In [1]:
! pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.0 MB/s eta 0:00:00


In [2]:
! pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=2df5ff4b27c5c6d20855ed17ed73c70612acccf91cc101c8e20e9c4e72e3b60a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


## Generate implicit event

In [3]:
prompt="Let's find out an event that is unmentioned but can be inferred from the story and the temporal relation between the two events are not deterministic. The new event should not be longer than ten words and include only one verb.\nStory: Tim's tooth was hurting like crazy. He could barely eat or drink. His dentist took a look around in his mouth. One of his teeth was rotten. Once the tooth was pulled, Tim felt fine.\nWhat is an event that is unmentioned but has some role and can be inferred from the story?\nTim scheduled an appointment with his dentist\nIt can be inferred from Tim's tooth was hurting like crazy.\n###\nStory:  Lily went to a nice restaurant. She ordered a steak. To her dismay the steak was rare. Lily was rather upset. She had to send it back.\nWhat is an event that is unmentioned but has some role and can be inferred from the story?\nLily ordered something else\nIt can be inferred from Lily was rather upset."
print(prompt)

Let's find out an event that is unmentioned but can be inferred from the story and the temporal relation between the two events are not deterministic. The new event should not be longer than ten words and include only one verb.
Story: Tim's tooth was hurting like crazy. He could barely eat or drink. His dentist took a look around in his mouth. One of his teeth was rotten. Once the tooth was pulled, Tim felt fine.
What is an event that is unmentioned but has some role and can be inferred from the story?
Tim scheduled an appointment with his dentist
It can be inferred from Tim's tooth was hurting like crazy.
###
Story:  Lily went to a nice restaurant. She ordered a steak. To her dismay the steak was rare. Lily was rather upset. She had to send it back.
What is an event that is unmentioned but has some role and can be inferred from the story?
Lily ordered something else
It can be inferred from Lily was rather upset.


In [8]:
import pandas as pd
final_prompt = []
with open('../data/gpt3.5/story_100_sample.txt') as f:
  for line in f:
    final_output_origin_1 = prompt+'\n###\nStory: '+line+'What is an event that is unmentioned but has some role and can be inferred from the story?'
    final_prompt.append([final_output_origin_1,line.strip()] )


In [9]:
import os
import openai

openai.api_key = ""

for i in range(len(final_prompt)):
    response = openai.Completion.create(
    model="text-davinci-002",
    prompt=final_prompt[i][0],
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    answer = response['choices'][0]['text']
    final_prompt[i].append(answer)

df = pd.DataFrame(final_prompt)
df.to_csv('gpt3.5_roc_raw_implicit_event.csv',index= None)

In [10]:
# filter generared events that are similar to the oiginal context
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


final_excel = []
for i in range(len(df)):
  story = df.iloc[i,1]
  event = df.iloc[i,2].split('\n')
  if len(event) == 3 and 'It can be inferred from' in event[2]:
    event1 = event[1]
    if len(event[2].split('It can be inferred from ')) >=2 : 
      event2 = event[2].split('It can be inferred from ')[1]

      max_score = 0
      for t in story.split('.'):
        embedding_1= model.encode(event1, convert_to_tensor=True)
        embedding_2 = model.encode(t, convert_to_tensor=True)

        score = util.pytorch_cos_sim(embedding_1, embedding_2)
        
        if score.tolist()[0][0] > 0.9:
          max_score = score.tolist()[0][0]
          print(story)
          print(event[1],event[2])
          print(score.tolist()[0][0])
      
      if max_score == 0:
        implicit = event1
        final_excel.append([story, event1, event2, implicit])

df = pd.DataFrame(final_excel)
df.columns = ['passage','event1','event2','implicitevent']
df.to_csv('gpt3.5_final_implicit_event.csv',index=False)

Bryan needed to write an essay for his class. Bryan went to the library and checked out a lot of books. He also went online and searched through google for information. He spent a few days doing research. After a few days, he wrote his essay and he did a good job.
Bryan read the books he checked out from the library It can be inferred from Bryan went to the library and checked out a lot of books.
0.94793701171875


# Generate first sentence with explanation

## Generate raw data with GPT-3.5

In [11]:
# More Prompts can be referred to in the paper
prompt="Let’s add a sentence as the first sentence of the paragraph to let the statement more likely to hold true and explain why.\nParagraph: I have always been attracted to Hispanic men. That said, my first huge crush was on a Mexican. I was in love with him for two years. After two years, I realized I was wasting my time and idolizing him. Without any real sense of closure, I decided to pull my heart away.\nStatement: I felt lonely  before I decided to pull my heart away\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?\nMy first boyfriend was a long distance relationship and even when we were together, I felt like he ignored me. \nThis makes the statement true because it implies her first crush's behavior made her feel lonely.\n###\nParagraph: I have always been attracted to Hispanic men. That said, my first huge crush was on a Mexican. I was in love with him for two years. After two years, I realized I was wasting my time and idolizing him. Without any real sense of closure, I decided to pull my heart away.\nStatement: I felt lonely after I decided to pull my heart away\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?\nI remember always getting a great deal of attention in all my relationships. \nThis makes the statement true because it implies she felt lonely because she ended a relationship and she can no longer get attention from her boyfriend.\n###\nParagraph: Tim's tooth was hurting like crazy. He could barely eat or drink. His dentist took a look around in his mouth. One of his teeth was rotten. Once the tooth was pulled, Tim felt fine.\nStatement: Tim scheduled an appointment with his dentist after his tooth was hurting like crazy\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?\nTim's tooth was usually perfect, so he did not often go to see the dentist.\nThis makes the statement true because it implies Tim did not have a regular appointment with his dentist and the reason why he scheduled an appointment with his dentist was because his tooth was hurting like crazy.\n###\nParagraph: Tim's tooth was hurting like crazy. He could barely eat or drink. His dentist took a look around in his mouth. One of his teeth was rotten. Once the tooth was pulled, Tim felt fine.\nStatement: Tim scheduled an appointment with his dentist before his tooth was hurting like crazy\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?\nTim always met his dentist regularly.\nThis makes the statement true because it implies Tim may have already had a regular appointment with his dentist before his tooth was hurting like crazy.\n###\nParagraph: Chuck was hanging out with some friends at a bar. They mentioned that they were moving soon. Chuck offered to help them move their things. The team worked together and got the move done quickly. They were so grateful that they invited him to stay for dinner.\nStatement: Chuck wanted to be helpful starts before Chuck offer to help them move their things\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?\nChuck is the kind of person that always wants to help out.\nThis makes the statement true because it implies Chuck's desire to help his friends move their things was because he is naturally helpful.\n###\nParagraph: Chuck was hanging out with some friends at a bar. They mentioned that they were moving soon. Chuck offered to help them move their things. The team worked together and got the move done quickly. They were so grateful that they invited him to stay for dinner.\nStatement: Chuck wanted to be helpful starts after Chuck offer to help them move their things\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?\nChuck often found himself reluctant to do thing, but grateful afterward that he did.\nThis makes the statement true because if Chuck was reluctant, he might not have truly felt like being helpful until after he offered to help and was grateful afterward."
print(prompt)

Let’s add a sentence as the first sentence of the paragraph to let the statement more likely to hold true and explain why.
Paragraph: I have always been attracted to Hispanic men. That said, my first huge crush was on a Mexican. I was in love with him for two years. After two years, I realized I was wasting my time and idolizing him. Without any real sense of closure, I decided to pull my heart away.
Statement: I felt lonely  before I decided to pull my heart away
Add what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?
My first boyfriend was a long distance relationship and even when we were together, I felt like he ignored me. 
This makes the statement true because it implies her first crush's behavior made her feel lonely.
###
Paragraph: I have always been attracted to Hispanic men. That said, my first huge crush was on a Mexican. I was in love with him for two years. After two years, I realized I was wasting my time and idolizing h

In [12]:
import pandas as pd
final_prompt = []
import pandas as pd
df = pd.read_csv('/content/gpt3.5_final_implicit_event.csv') #/content/gpt3_new_event.csv

for i in range(len(df)):
  origin_story = df.iloc[i,0]
  event_1 = df.iloc[i,1].replace('.','')
  event_2 =  df.iloc[i,2].replace('.','')
  if ',' in event_2:
    event_2 = event_2.split(',')[0]
  elif 'and' in event_2:
    event_2 = event_2.split('and')[0]
  event1 = event_1 + ' starts before ' + event_2
  event2 = event_1 + ' starts after ' + event_2
  final_output_origin_1 = prompt+'\n###\nParagraph: '+origin_story+'\nStatement: '+event1+ '\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?'
  final_output_origin_2 = prompt+'\n###\nParagraph: '+origin_story+'\nStatement: '+event2+ '\nAdd what sentence as the first sentence of the paragraph and why is the statement more likely to hold true?'
  final_prompt.append([final_output_origin_1,origin_story,event1] )
  final_prompt.append([final_output_origin_2,origin_story,event2])

In [16]:
import os
import openai

openai.api_key = ""

for i in range(len(final_prompt)):
  response = openai.Completion.create(
  model="text-davinci-002",
  prompt=final_prompt[i][0],
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
  )
  answer = response['choices'][0]['text']
  final_prompt[i].append(answer)

In [17]:
import pandas as pd
df = pd.DataFrame(final_prompt)
df.to_csv('gpt3.5_generated_first_sentence_w_explanation.csv',index=False)

## Rule based filter

In [18]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

df = pd.read_csv('/content/gpt3.5_generated_first_sentence_w_explanation.csv')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [19]:
import pandas as pd
item_dict = dict()
for i in range(len(df)):
  if '\n' in df.iloc[i,3]:
    temp = df.iloc[i,3].split('\n')

    explanation = ''
    addition = ''
    for sent in temp:
      if sent == '': continue
      if 'This makes the statement true because' in sent:
        explanation = sent.split('This makes the statement true because')[1][1:].capitalize()
      elif 'This sentence implies that' in sent:
        explanation = sent.split('This sentence implies that')[1][1:].capitalize()
      elif 'It implies that' in sent:
        explanation = sent.split('It implies that')[1][1:].capitalize()
      else:
        addition = sent
        if addition[0] == ' ':
          addition = addition[1:]

    if ", which would make it more likely " in explanation:
      if '' in explanation.split(", which would make it more likely "):
        explanation = explanation.split(", which would make it more likely ")[0] + '.'


    origin_story = df.iloc[i,1]
    final_story = '[' + addition + '] ' +  origin_story
    event = df.iloc[i,2]
    final_output = 'event: '+event + ' story: '+ final_story + ' explanation: '+ explanation + '	answer: positive' + '\n'
    item_dict[final_output] = [origin_story, event, addition, explanation, final_story]

final_delete_set = set()
df = pd.DataFrame(item_dict).T

In [20]:
# additional first sentence with explanation 
all_score = []
for i in range(len(df)):
  event = df.iloc[i,1]
  origin_story = df.iloc[i,0]
  additional_sentence = df.iloc[i,2]
  explanation = df.iloc[i,3]

  if 'This sentence implies that' in explanation:
    explanation = explanation.split('This sentence implies that')[1][1:]
  elif 'The first sentence implies that' in explanation:
    explanation = explanation.split('The first sentence implies that')[1][1:]
  elif 'This makes the statement true because the first sentence implies that' in explanation:
    explanation = explanation.split('This makes the statement true because the first sentence implies that')[1][1:]
  elif 'This makes the statement true because the first sentence implies' in explanation:
    explanation = explanation.split('This makes the statement true because the first sentence implies')[1][1:]
  elif 'This makes the statement true because it implies that' in explanation:
    explanation = explanation.split('This makes the statement true because it implies that')[1][1:]
  elif 'This makes the statement true because it implies' in explanation:
    explanation = explanation.split('This makes the statement true because it implies')[1][1:]
  elif 'This makes the statement true because' in explanation:
    explanation = explanation.split('This makes the statement true because')[1][1:]

  embedding_1= model.encode(additional_sentence, convert_to_tensor=True)
  embedding_2 = model.encode(explanation, convert_to_tensor=True)
  score = util.pytorch_cos_sim(embedding_1, embedding_2)

  if score.tolist()[0][0] >= 0.9:
    final_delete_set.add(i)



In [21]:
# additional first sentence with paragraph
def paragraph_split(origin_story):
  final_story_split=[]
  tag = ''
  for sent in origin_story.split('. '):
    if tag == '':
      if len(sent) <= 15:
        tag = sent
      else:
        final_story_split.append(sent)
    elif tag != '':
      if len(sent) <= 15:
        tag = tag+'. '+sent
      else:
        final_story_split.append(tag+'. '+sent)
        tag = ''
  return final_story_split

all_score = []
for i in range(len(df)):
  event = df.iloc[i,1]
  origin_story = df.iloc[i,0]
  additional_sentence = df.iloc[i,2]
  final_story_split = paragraph_split(origin_story)
  
  for sent in final_story_split:
    embedding_1= model.encode(additional_sentence, convert_to_tensor=True)
    embedding_2 = model.encode(sent, convert_to_tensor=True)
    score = util.pytorch_cos_sim(embedding_1, embedding_2)

    if score.tolist()[0][0] >= 0.95:
      final_delete_set.add(i)



In [22]:
# event with explanation
final_explanation_list = []
explanation_all_score = []
all_score = []
for i in range(len(df)):
  explanation = df.iloc[i,3]
  event = df.iloc[i,1]
  explanation_split = paragraph_split(explanation)
  delete_explanation = []
  tag = 0 
  for sent in explanation_split:
    embedding_1= model.encode(sent, convert_to_tensor=True)
    embedding_2 = model.encode(event, convert_to_tensor=True)
    score = util.pytorch_cos_sim(embedding_1, embedding_2)

    explanation_all_score.append([event, sent, score.tolist()[0][0]])
    all_score.append(score.tolist()[0][0])

    if score >= 0.9:
      delete_explanation.append(sent)
      tag = 1

  final_explanation = ''
  for j,sent in enumerate(explanation_split):
    if sent in delete_explanation:
      continue
    else:
      if j == len(explanation_split)-1 :
        final_explanation += sent
      else:
        final_explanation += sent + '. '
  
  if tag == 1:
    if final_explanation == '':
      final_delete_set.add(i) 
    elif final_explanation[-1] == ' ':
      final_explanation = final_explanation[0:-1]
    df.iloc[i,4] = final_explanation
    final_explanation_list.append(final_explanation)
  else:
    final_explanation_list.append(explanation)
  

## Get GPT-3.5 output data

In [24]:
save_name = 'gpt3.5_generated_first_sentence'
name = save_name + '_with_explanation'
f = open(name+'.txt','w')
name_1 = save_name + '_without_explanation'
f_1 = open(name_1+'.txt','w')
f_2 = open(save_name +'_explanation_verifier.txt','w')
item_dict_1 = dict()
for i in range(len(df)):
  if i not in final_delete_set:
    explanation = df.iloc[i,3]
    event = df.iloc[i,1]
    origin_story = df.iloc[i,0]
    additional_sentence = df.iloc[i,2]


    if 'The additional sentence implies that' in explanation:
      explanation = explanation.split('The additional sentence implies that')[1][1:]
    elif 'The additional sentence implies' in explanation:
      explanation = explanation.split('The additional sentence implies')[1][1:]
   
    if additional_sentence[-1] == ' ':
      additional_sentence = additional_sentence[0:-1]


    final_story = '[' + additional_sentence + '] ' +  origin_story
    final_origin_output = 'event: '+event +' story: '+ origin_story + '\n'
    final_output = 'event: '+event + ' story: '+ final_story + ' explanation: '+ explanation + '\n'
    final_output_1 = 'event: '+event + ' story: '+ final_story + '\n'


    if 'after' in event:
      final_reverse_origin_output = 'event: '+event.replace('after','before') +' story: '+ origin_story + '\n'
      final_reverse_output = 'event: '+event.replace('after','before') +' story: '+ final_story + ' explanation: '+ explanation+ '\n'
      final_reverse_output_1 = 'event: '+event.replace('after','before') + ' story: '+ final_story + '\n'
    elif 'before' in event:
      final_reverse_origin_output = 'event: '+event.replace('before','after') +' story: '+ origin_story + '\n'
      final_reverse_output = 'event: '+event.replace('before','after') + ' story: '+ final_story +' explanation: '+ explanation + '\n'
      final_reverse_output_1 = 'event: '+event.replace('before','after') + ' story: '+ final_story + '\n'


    f.write(final_origin_output)
    f.write(final_reverse_origin_output)
    f.write(final_output)
    f.write(final_reverse_output)

    f_1.write(final_origin_output)
    f_1.write(final_reverse_origin_output)
    f_1.write(final_output_1)
    f_1.write(final_reverse_output_1)


    final_story = '[' + additional_sentence+'] '+origin_story
    final_pos_output = 'event: '+event + ' story: '+ final_story + ' explanation: The sentence with brackets make the event more likely to hold true because '+ explanation + '\n'
    f_2.write(final_pos_output)

